<a href="https://colab.research.google.com/github/dalgual/aidatasci/blob/main/opentable_sentiment_helpful.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install torch
!pip install tensorflow

In [44]:
!pip install accelerate -U

In [45]:
!pip install transformers[torch]

In [46]:
import torch
torch.cuda.is_available()

False

In [47]:
!pip install datasets transformers huggingface_hub


In [48]:
# !apt-get install git-lfs


In [49]:
from huggingface_hub import notebook_login
notebook_login()


In [50]:
import pandas as pd

data_files = "https://raw.githubusercontent.com/dalgual/aidatasci/master/data/classification/opentable_review.csv"
#pd.options.display.max_rows = 9 #999

df = pd.read_csv(data_files) #'data.csv')

#print(df)

In [51]:
# Drop null data
df = df.dropna(subset=['content'])

In [52]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46349 entries, 0 to 46391
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  46349 non-null  int64 
 1   nick        46349 non-null  object
 2   score       46349 non-null  int64 
 3   content     46349 non-null  object
 4   date        46349 non-null  object
 5   helpful     46349 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.5+ MB


In [53]:
import numpy as np

#define conditions
conditions = [
    df['helpful']==0,
    df['helpful']>0
]

#define results
results = [0, 1]

#create new column based on conditions in column1 and column2
df['labels'] = np.select(conditions, results)


<ipython-input-53-9eee76ce843c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'] = np.select(conditions, results)


In [54]:
df

,Unnamed: 0,nick,score,content,date,helpful,labels
0,45435,A Google user,5,I just found and reserved a table for dinner t...,2009-09-14 17:21:18,0,0
1,45541,A Google user,3,Good start takes up too much space and leaves ...,2009-09-14 21:00:11,0,0
2,45925,A Google user,5,Koo app . Very useable,2009-09-15 3:28:32,0,0
3,45446,A Google user,4,"Awesome, just waaaaaayyyyy too big for this ki...",2009-09-15 15:43:21,0,0
4,45398,A Google user,5,I luv that OpenTable is so responsive to reque...,2009-09-15 17:38:47,0,0
...,...,...,...,...,...,...,...
46387,526,Stephanie Magathan-Droubay,5,Makes reservations fast and easy,2023-01-15 22:23:49,0,0
46388,170,Kent Weber,4,Very useful app when exploring a new area. Jus...,2023-01-15 23:57:52,0,0
46389,3525,Tim Bolognone,5,Easy to use.,2023-01-16 1:02:59,0,0
46390,238,Neil Stout,4,Easy to determine availability and to book a r...,2023-01-16 2:36:28,0,0


In [55]:
df[df['labels'] == 1]

,Unnamed: 0,nick,score,content,date,helpful,labels
232,45693,A Google user,5,"Cool app., needs better search capabilities. ....",2010-05-18 2:09:24,1,1
264,44832,A Google user,4,Love Open Table and this is a good app. Add th...,2010-06-30 2:24:48,1,1
324,44664,A Google user,5,Great idea. Booked a reservation for my annive...,2010-08-12 17:47:12,1,1
325,44542,A Google user,3,The menu doesn't scroll correctly on my HTC In...,2010-08-12 22:01:33,1,1
335,44537,A Google user,4,"Very useful app, especially here in Las Vegas!",2010-08-15 3:07:12,1,1
...,...,...,...,...,...,...,...
46298,3,JT Sween,3,This app has a lot of trouble identifying know...,2023-01-09 18:36:34,24,1
46338,14992,A Google user,1,This app is *still* so full of fail. How is th...,2023-01-12 4:06:59,1,1
46357,99,Tim,4,Great for finding and reserving restaurants al...,2023-01-13 23:12:02,1,1
46373,78,D. Coven,3,I would say is that most restaurants are vastl...,2023-01-14 20:51:14,3,1


In [56]:
df1 = df.sample(frac = 0.7, random_state = 200)

In [57]:
df1


,Unnamed: 0,nick,score,content,date,helpful,labels
2396,42831,A Google user,5,great app.,2012-02-14 14:50:31,1,1
3443,45090,A Google user,4,"Great app to have, would give it 5 stars if it...",2012-11-05 0:10:38,0,0
24578,21433,A Google user,3,"Not sure how new it is, ive just downloaded it...",2018-01-17 2:05:12,0,0
7237,39219,A Google user,5,Cool!,2015-08-21 22:39:27,0,0
1797,43322,A Google user,4,Easy to use. Quickly find restaurants and rese...,2011-09-18 14:16:51,0,0
...,...,...,...,...,...,...,...
5000,45065,A Google user,2,It neither shows you nor allows you to search ...,2014-06-06 23:43:04,0,0
10299,34976,A Google user,1,Can't make reservations through the app. Only ...,2015-12-25 4:46:43,0,0
19625,26423,A Google user,5,"Extremely handy managing reservations, using i...",2017-03-13 23:21:46,0,0
10432,35205,A Google user,4,Just need to represent more restaurants in my ...,2015-12-30 6:09:29,0,0


In [58]:
from datasets import load_dataset
from datasets import Dataset

# dataset = load_dataset("csv", data_files=data_files, split="train")
dataset = Dataset.from_pandas(df)


In [59]:
dataset

Dataset({
    features: ['Unnamed: 0', 'nick', 'score', 'content', 'date', 'helpful', 'labels', '__index_level_0__'],
    num_rows: 46349
})

In [60]:
# https://discuss.huggingface.co/t/how-to-split-hugging-face-dataset-to-train-and-test/20885/3
train_ds=dataset.train_test_split(test_size=0.3, shuffle=True) #, stratify_by_column='Sentiment')
print(train_ds)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'content', 'date', 'helpful', 'labels', '__index_level_0__'],
        num_rows: 32444
    })
    test: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'content', 'date', 'helpful', 'labels', '__index_level_0__'],
        num_rows: 13905
    })
})


In [61]:
#train_ds = train_ds.rename_column("Sentiment", "labels")
train_ds = train_ds.rename_column("content", "text")
print(train_ds)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
        num_rows: 32444
    })
    test: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
        num_rows: 13905
    })
})


In [62]:
train_ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
        num_rows: 32444
    })
    test: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
        num_rows: 13905
    })
})

In [63]:
small_train_dataset = train_ds["train"].shuffle(seed=42).select([i for i in list(range(32444))])
small_test_dataset = train_ds["test"].shuffle(seed=42).select([i for i in list(range(13905))])


In [64]:
small_train_dataset

Dataset({
    features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
    num_rows: 32444
})

In [65]:
small_test_dataset

Dataset({
    features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
    num_rows: 13905
})

In [66]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [67]:
def preprocess_function(examples):
   #return tokenizer(examples["SentimentText"], truncation=True)
   return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/32444 [00:00<?, ? examples/s]

Map:   0%|          | 0/13905 [00:00<?, ? examples/s]

In [68]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [69]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
   load_prec = load_metric("precision")
   load_recall = load_metric("recall")
   #load_auc = load_metric("roc_auc")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   prec = load_prec.compute(predictions=predictions, references=labels)["precision"]
   recall = load_recall.compute(predictions=predictions, references=labels)["recall"]
   #auc = load_auc.compute(predictions=predictions, references=labels)["roc_auc"]

   #return {"roc_auc":auc, "accuracy": accuracy, "f1": f1, "precision":prec, "recall":recall}
   return {"accuracy": accuracy, "f1": f1, "precision":prec, "recall":recall}



In [71]:
#!pip install accelerate -U

In [72]:
#!pip install transformers[torch]

In [73]:
from transformers import TrainingArguments, Trainer


#repo_name = "finetuning-sentiment-model-3000-samples-jwoo5"
repo_name = "semadalg/finetuning-opetable-model"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)




In [74]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [75]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.360600
1000,0.326300
1500,0.336300
2000,0.338200
2500,0.324500
3000,0.319900
3500,0.313500
4000,0.320700


TrainOutput(global_step=4056, training_loss=0.3306237423913719, metrics={'train_runtime': 15131.9081, 'train_samples_per_second': 4.288, 'train_steps_per_second': 0.268, 'total_flos': 949079236095744.0, 'train_loss': 0.3306237423913719, 'epoch': 2.0})

In [76]:
trainer.evaluate()

{'eval_loss': 0.3364441394805908,
 'eval_accuracy': 0.8706220783890687,
 'eval_f1': 0.28639428798095995,
 'eval_precision': 0.5135135135135135,
 'eval_recall': 0.19856985698569857,
 'eval_runtime': 946.493,
 'eval_samples_per_second': 14.691,
 'eval_steps_per_second': 0.919,
 'epoch': 2.0}

In [1]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 10.7 MB/s eta 0:00:00


In [3]:
import evaluate

roc_auc_score = evaluate.load("roc_auc")


In [4]:
accuracy = evaluate.load("accuracy")
accuracy.description

'\nAccuracy is the proportion of correct predictions among the total number of cases processed. It can be computed with:\nAccuracy = (TP + TN) / (TP + TN + FP + FN)\n Where:\nTP: True positive\nTN: True negative\nFP: False positive\nFN: False negative\n'

In [5]:
clf_metrics = evaluate.combine(["roc_auc", "accuracy", "f1", "precision", "recall"])

In [ ]:
clf_metrics.compute(predictions=predictions, references=labels)


In [ ]:
# predictions=predictions, references=labels)["roc_auc"]
results = roc_auc_score.compute(references=labels, prediction_scores=predictions)
print(round(results['roc_auc'], 2))

In [ ]:
trainer.push_to_hub()


In [ ]:
from transformers import pipeline

sentiment_model = pipeline(model="semadalg/finetuning-opetable-model")
sentiment_model(["I love this move", "This movie sucks!"])


In [ ]:
predictions = trainer.predict(tokenized_test)
# y_pred = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
predictions.predictions.shape

In [ ]:
predictions.label_ids.shape

In [ ]:
predictions

In [ ]:
df_preds=pd.DataFrame(predictions.predictions)
df_preds.apply(lambda row : np.exp(row[0])/(np.exp(row[0])+np.exp(row[1])),axis=1)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)



In [ ]:
probs = np.exp(preds[:])

In [ ]:
preds

In [ ]:
preds2 = np.argmax(predictions.predictions, axis=1)

In [ ]:
preds2

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids, zero_division=0)

## References
1. Getting Started with Sentiment Analysis using Python, https://huggingface.co/blog/sentiment-analysis-python
1. The model did not return a loss from the inputs, only the following keys: logits, https://discuss.huggingface.co/t/the-model-did-not-return-a-loss-from-the-inputs-only-the-following-keys-logits-for-reference-the-inputs-it-received-are-input-values/25420/3
